본 강좌에서는 텐서보드(TensorBoard)를 사용해서 학습 과정을 시각화하는 방법에 대해서 알아보겠습니다.

---

### 학습 과정을 시각화하는 이유

저는 이렇게 이해했습니다. 어차피 학습과정이니 시험치는 것에 비유해봤습니다.

- 모의고사 1부에 100문항이 있다고 가정합니다.
- epoch는 같은 모의고사를 몇 번 풀 것이냐 입니다. epoch가 10이면 같은 모의고사지를 10번 푸는 것과 동일합니다. 처음 epoch를 접할 때는 동일한 데이터셋을 여러번 사용하는 것이 도움이 돼?라고 생각했는데, 같은 모의고사지를 여러번 푼다고 생각해보니, 점점 학습이 될 것 같더군요.
- batch size는 몇 문항을 보고 답을 맞춰볼까 입니다. 실전처럼 하는 사람은 전 문항을 다 풀고 난 다음 답을 맞춰서 오답정리를 하는 사람도 있고 성격이 급한 사람은 한 문제 풀고 답 맞추는 사람도 있겠죠. 처음엔 이것도 학습에 무슨영향을 미칠까 의아해 했는데, 같은 모의고사지에 비슷한 문제가 있을 경우 다 풀어보고 맞춘 사람은 한 문제 틀리면 다른 유사문데들도 다 틀리겠지만, 한 문항보고 오답정리한 사람은 첫 문제를 틀리더라도 다른 남은 문제들은 맞출 확률이 높아지겠죠. 
- 그밖에 train, validation, test셋을 왜 나누어야 하는 지도 모의고사, 실전 모의고사, 실제 수능 등으로 생각해보니 당연히 그렇게 해야겠구나 생각이 들더군요.

비유가 이상하거나 제가 잘 못 이해하고 있다면 지적해주세요~


검증셋은 학습에 영향을 미치지 않는다.
학습은 문제와 답을 준 상태에서 학생이 문제를 풀고 답을 맞춰보는 과정에서 이루어 집니다. 이 때 학생이 아하~ 하면서 이해하는 것이 weight를 업데이트 하는 것입니다.

---

### 케라스 전용 시각화 툴

[검증셋 이야기]
데이터셋에 관한 글을 읽다가 제가 얼마나 무지했던가?를 깨닫고 정리해봤습니다. 기초적인 내용이라서 다들 잘 아시겠지만 입문하시는 분들에게는 도움이 되실까 올려봅니다.
# 처음엔
- 검증셋이 훈련 시에 사용되기에 back-propagation 시 weight에 영향을 미치는 줄 알았습니다.
- 같은 이유로 k-fold cross validation 하면 좀 더 검증된 모델이 훈련되는 줄 알았습니다.
- 테스트셋으로 평가해서 좋은 모델을 선택했습니다.
# 하지만 
- 검증셋은 훈련 시 모델에 영향을 미치지 않습니다. 단 보통 훈련셋에서 일부를 검증셋으로 사용하기 때문에 검증셋의 선택을 바꾸면, 훈련셋이 바뀌기 때문에 영향을 미칠 수는 있습니다만 이것은 검증셋 때문이라기 보다 훈련셋의 변화로 영향을 미치는 것입니다. 또한 과적합을 피하기 위해 조기종료를 할 경우 검증셋의 에러율을 이용하기 때문에 이때는 모델 훈련에 영향을 미칠 수 있겠네요.
- k-fold cross validation 시 검증셋이 바뀔 때 마다(iteration시 마다) 학습된 모델을 사용하는 것이 아니라, 모델의 weight는 초기화해야하고, 각 fold에서 나오는 검증셋의 에러율을 평균하여 해당 모델의 에러율을 판정합니다. 하나의 검증셋보다 여러 개의 검증셋으로 검증하고, 각 검증 시에는 처음부터 훈련을 시켜야 검증에 대한 신뢰성이 높아집니다.
- 모델 선택은 테스트셋의 평가가 아닌 검증셋의 평가로 이루어집니다. 모델 선택이란 여러 후보 모델이 있을 경우, 검증셋을 통해서 가장 낮은 에러를 가진 모델을 최종 모델로 선택하는 것을 말합니다. 여러 후보 모델이란 의미는 SVM, CNN 등 모델 자체가 다른 것도 있겠지만, 같은 모델이라고 하더라도 설정된 파라미터가 다른 경우에도 적용됩니다. 이를 하이퍼파라미터 튜닝이라고 하는 데, CNN인 경우에는 필터 크기, 개수 등이 있을 수 있겠네요. 즉 필터의 설정을 바꿔가면서 훈련 시키고, 그 중 좋은 파라미터를 선택할 때 사용되는 것이 검증셋입니다. 그러니 k-fold cross validation을 사용하게되면, 튜닝된 모델 선택 시에 좀 더 검증된 모델을 선택할 수 있는 것입니다. GridSearchCV() 함수 예제를 보니, CNN에서의 하이퍼파라미터을 찾는 것이 상상이 되더군요. (필터 크기를 바꿔가면서, 필터 개수를 바꿔가면서, epoch를 바꿔가면서 등등 가장 좋은 모델을 선택하는 코드를 봤습니다만 그저 상상에 그쳤습니다. 해보기엔 지금 장비로 엄두가 나지 않네요. )
# 아직 확신이 들지 않는 것은
- 검증셋을 통해 모델이 선택되면 그대로 사용하는 것이 아니라 모델을 전체 훈련셋으로 다시 훈련을 시켜라는 코멘트도 봤는데, 과적합이 되지 않는 epoch에서 전체 훈련셋으로 훈련할 경우, 추가되는 훈련셋(기존에 검증셋으로 사용했던) 때문에 과적합이 생기는 건 아닌 지 염려스럽네요. (미비할 것이라고 추측은 됩니다만)
- 훈련셋이 적을 경우 데이터 업샘플링(회전, 확대/축소, 각종 변형)을 하는 데, 검증셋에도 적용해도 될까요? k-fold cross validation을 하게 되면 업샘플링된 훈련셋에서 검증셋을 선정하기 때문에 검증셋도 업샘플링된 데이터로 구성이 되긴 하는데, 바람직한지는 잘 모르겠습니다. 검증셋은 업샘플링 등의 과정이 없이 최대한 원본 그대로를 유지해야 좀 더 신뢰성이 높을 것 같습니다.
잘못된 정보가 있다면 코멘트 부탁드리겠습니다.

---

### 텐서보드(TensorBoard)란

Dong-Hyun Lee Competition처럼 결과에 매우 민감한 상황에는 valid set으로 hyperparameter tuning을 한 후에 그걸 train set에 넣어 다시 training하는 것이 도움이 될 것 같습니다만, training sample수가 충분한 경우에는 구지 그렇게까지 할 필요가 있을까 싶습니다. / upsampling 과정이 reasonable 하다면 upsampling test set도 나름 의미가 있다고 생각이 듭니다만, 다른 모델과 정확한 비교를 하기 위해서는 완전히 같은 test set을 이용해야 할 것 같습니다.
좋아요 취소 · 답글 달기 · 2 · 2016년 8월 22일 오전 2:22 · 수정됨
김태영
김태영 의견 감사합니다~!
좋아요 · 답글 달기 · 2016년 8월 22일 오전 6:36


문창기 제 경우 보행자 검출시 초기 훈련 set을 육안으로 변별력 있게 잘 정제한 후 훈련된 분류기를 테스트 set에 적용하여 나온 false positive와 false negative를 retraining하는 방식으로 분류기의 성능 강화를 진행중에 있습니다.
좋아요 취소 · 답글 달기 · 1 · 2016년 8월 22일 오전 8:24
김태영
김태영 성능 강화에 좋은 아이디어이신것 같습니다. 테스트셋을 통해 FP, FN의 데이터를 훈련시킨 다음에는 테스트셋에서 제외 시키는 건가요? 훈련시킨 데이터가 훈련셋에도 있고 테스트셋에도 있으면 과적합이 될 것 같습니다.
좋아요 · 답글 달기 · 2016년 8월 22일 오후 5:04
문창기
문창기 김태영 분류기의 일반화 능력이 어느정도 생긴 다음부터는 훈련 셋과 테스트 셋이 중복되더라도 오버피팅은 발생하지 않았습니다.
좋아요 취소 · 답글 달기 · 1 · 2016년 8월 22일 오후 10:11

문창기 분류기의 정확성도 중요하지만 속도를 고려해야 하기에 훈련 및 테스트 셋에 일정부분 사람의 개입을 의도적으로 해주고 있습니다.
좋아요 취소 · 답글 달기 · 2 · 2016년 8월 22일 오전 8:28
김태영
김태영 어떤 식으로 사람 개입이 되는지 궁금합니다. 저도 사람이 판정한 훈련셋의 신뢰도를 높이기 위해, 모델 결과 중 FP, FN을 다시 사람에게 보여줘서 재판독 하는 과정을 넣어서 내가 훈련시킨 모델을 이용하여 훈련셋 정제에 사용하고자 하는데, 이런식의 개입인가요?
좋아요 · 답글 달기 · 2016년 8월 22일 오후 5:06
문창기
문창기 김태영 db를 구성하는 방법에 있어서 개입을 말씀 드렸습니다. Occlusion 도 추가시키고, 랜덤 노이즈와 랜덤 블러링도 살짝씩 추가시키구요. 물론 초기부터 적용하는게 아니라, 적어도 주어진 테스트 셋에 대해서 일반화 능력이 어느정도 만들어진 다음부터 테스트 셋을 위와 같이 인위적 개입을 통해 강화시키고 있습니다.
좋아요 취소 · 답글 달기 · 1 · 2016년 8월 22일 오후 10:17


Dohyeong Kim 전체 데이터 세트를 k-fold cross-validation을 이용하여 나누고 grid search를 이용하여 파라미터를 바꾸어 가면서 계산하여 최종적으로 나오는 최적 모델은 하나일텐데 이 모델을 다른 데이터 세트로 훈련 한다는 말인가요?
좋아요 취소 · 답글 달기 · 1 · 2016년 8월 22일 오후 7:02 · 수정됨
김태영
김태영 제가 본 코멘트에서는 훈련셋이 부족해서 최적모델이 선택되면 같은 파라미터로 검증셋까지 훈련시키는 것 같았습니다. 하지만 말씀하신 것처럼 최적 모델이 나오면 더이상 업데이트를 안 하는 것이 맞을 것 같습니다.
좋아요 · 답글 달기 · 1 · 2016년 8월 23일 오전 8:26 · 수정됨


박세진 가장 낮은 에러를 가지는 모델이 항상 좋지는 않은것같습니다
또 k fold나 subsampling이 만드는 불균일성 문제도 있는것 같고요
Examplar svm형태의 hinge loss를 가지는 모델의 앙상블도 고려해봄직한데
데이터개수의 모델을 만들어야 한다는 현실적문제도 있고요
저도 이걸로 계속 고민중인데 아직 정답근처도 못간것 같습니다
가장 큰 문제는 negative data의 분포를 모델링하기가 너무 어렵다는 것
fp,fn정제 및 stage 마다 갱신하는 학습방법도 어느정도 데이터에선 효과적이었습니다

---

### 케라스와 텐서보드를 연동하는 법

Tensorboard basic visualizations.

    keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False)

This callback writes a log for TensorBoard, which allows you to visualize dynamic graphs of your training and test metrics, as well as activation histograms for the different layers in your model.

TensorBoard is a visualization tool provided with TensorFlow.

If you have installed TensorFlow with pip, you should be able to launch TensorBoard from the command line:

tensorboard --logdir=/full_path_to_your_logs
You can find more information about TensorBoard - __here.

Arguments

* log_dir : the path of the directory where to save the log files to be parsed by Tensorboard
* histogram_freq: frequency (in epochs) at which to compute activation histograms for the layers of the model. If set to 0, histograms won't be computed.
* write_graph: whether to visualize the graph in Tensorboard. The log file can become quite large when write_graph is set to True.


RuntimeError: TensorBoard callback only works with the TensorFlow backend.

In [1]:
import numpy as np

# 랜덤시드 고정시키기
np.random.seed(5)

from keras.preprocessing.image import ImageDataGenerator

# 데이터셋 불러오기
train_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
        'warehouse/handwriting_shape/train',
        target_size=(24, 24),
        batch_size=3,
        class_mode='categorical')

validation_datagen = ImageDataGenerator()

validation_generator = validation_datagen.flow_from_directory(
        'warehouse/handwriting_shape/validation',
        target_size=(24, 24),    
        batch_size=3,
        class_mode='categorical')

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D

# 모델 구성하기
model = Sequential()
model.add(Convolution2D(12, 3, 3, border_mode='same', input_shape=(3, 24, 24), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(2, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(3, 2, 2, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='softmax'))

# 모델 엮기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

from keras.callbacks import TensorBoard

# 텐서보드 연동하기
tensorboard_callback = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=True)

# 모델 학습시키기
model.fit_generator(
        train_generator,
        samples_per_epoch=45,
        nb_epoch=200,
        validation_data=validation_generator,
        nb_val_samples=15,
        callbacks=[tensorboard_callback])

# 모델 평가하기
print("-- Evaluate --")

scores = model.evaluate_generator(
            validation_generator, 
            val_samples = 15)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 모델 예측하기
print("-- Predict --")

output = model.predict_generator(
            validation_generator, 
            val_samples = 15)

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

print(output)

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.
INFO:tensorflow:Summary name convolution2d_1_W:0 is illegal; using convolution2d_1_W_0 instead.


Using TensorFlow backend.
INFO:tensorflow:Summary name convolution2d_1_W:0 is illegal; using convolution2d_1_W_0 instead.


INFO:tensorflow:Summary name convolution2d_1_W:0 is illegal; using convolution2d_1_W_0 instead.


INFO:tensorflow:Summary name convolution2d_1_W:0 is illegal; using convolution2d_1_W_0 instead.


INFO:tensorflow:Summary name convolution2d_1_b:0 is illegal; using convolution2d_1_b_0 instead.


...

INFO:tensorflow:Summary name dense_2_b:0 is illegal; using dense_2_b_0 instead.


INFO:tensorflow:Summary name dense_2_b:0 is illegal; using dense_2_b_0 instead.


INFO:tensorflow:Summary name dense_2_b:0 is illegal; using dense_2_b_0 instead.


INFO:tensorflow:Summary name dense_2_b:0 is illegal; using dense_2_b_0 instead.


Epoch 1/200
45/45 [==============================] - 0s - loss: 8.5139 - acc: 0.3778 - val_loss: 7.7609 - val_acc: 0.4000
Epoch 2/200
45/45 [==============================] - 0s - loss: 7.5305 - acc: 0.4444 - val_loss: 6.6558 - val_acc: 0.4667
Epoch 3/200
45/45 [==============================] - 0s - loss: 5.5426 - acc: 0.6000 - val_loss: 5.6512 - val_acc: 0.5333
Epoch 4/200
45/45 [==============================] - 0s - loss: 5.4888 - acc: 0.6000 - val_loss: 5.7667 - val_acc: 0.5333
Epoch 5/200
45/45 [==============================] - 0s - loss: 5.1460 - acc: 0.6222 - val_loss: 5.2414 - val_acc: 0.3333
Epoch 6/200
45/45 [==============================] - 0s - loss: 2.1311 - acc: 0.4222 - val_loss: 1.6423 - val_acc: 0.2000
Epoch 7/200
45/45 [==============================] - 0s - loss: 1.0828 - acc: 0.3556 - val_loss: 1.1054 - val_acc: 0.2667
Epoch 8/200
45/45 [==============================] - 0s - loss: 1.0840 - acc: 0.3556 - val_loss: 1.0991 - val_acc: 0.3333
Epoch 9/200
45/45 [=====

---

### 결론

본 강좌에서는 이미지 분류 문제를 직접 정의해보고 데이터셋도 직접 만들어봤습니다. 이미지 분류 문제에 높은 성능을 보이고 있는 컨볼루션 신경망 모델을 이용하여 직접 만든 데이터셋으로 학습 및 평가를 해보았습니다. 학습 결과는 좋게 나왔지만 이 모델은 한 사람이 그린 것에 대해서만 학습이 되어 있어서 다른 사람에 그린 모양은 잘 분류를 못할 것 같습니다. 이후 강좌에서는 다른 사람이 그린 모양으로 평가해보고 어떻게 모델 성능을 높일 수 있을 지 알아보겠습니다.

그리고 실제 문제에 적용하기 전에 데이터셋을 직접 만들어보거나 좀 더 쉬운 문제로 추상화해서 프로토타이핑 하시는 것을 권장드립니다. 객담도말된 결핵 이미지 판별하는 모델을 만들 때, 결핵 이미지를 바로 사용하지 않고, MNIST의 손글씨 중 '1'과 '7'을 결핵이라고 보고, 나머지는 결핵이 아닌 것으로 학습시켜봤었습니다. 결핵균이 간균 (막대모양)이라 적절한 프로토타이핑이었습니다. 

---

### 같이 보기

* [강좌 목차](https://tykimos.github.io/Keras/2017/01/27/Keras_Lecture_Plan/)
* 이전 : [딥러닝 모델 이야기/컨볼루션 신경망 레이어 이야기](https://tykimos.github.io/Keras/2017/01/27/CNN_Layer_Talk/)
* 다음 : [딥러닝 모델 이야기/순환 신경망 레이어 이야기]